# Inverted index, queries and TF-IDF by hand in Python
In this notebook, I provide functions for cleaning, indexing and executing word and phrase queries, and show how to compute TF-IDF scores. All of this is illustrated on a toy collection of documents. We need very few external librairies:

In [1]:
import math
import nltk
import operator
from nltk import word_tokenize
stemmer = nltk.stem.PorterStemmer()

## Cleaning and indexing
The inverted index contains, for each unique term, the position(s) of the term in each document of the collection. It can be structured as a dictionary of dictionaries.

In [2]:
def clean_tokenize(doc):
    words = word_tokenize(doc.lower())
    return words

def index_one_doc(doc,to_stem):
    '''
    creates dict (tok,positions) for each tok in the document (as term list)
    '''
    tokpos = dict()
    for t_idx,tok in enumerate(doc):
        if to_stem:
           tok = stemmer.stem(tok)
        if tok in tokpos:
            tokpos[tok].append(t_idx)
        else:
            tokpos[tok] = [t_idx]
    return tokpos

Let's see what we get when indexing our toy corpus. We first clean the documents and inspect the positional lists for individual documents.

In [4]:
docs = ['The quick brown fox jumps over the lazy dog',
        'The brown quick fox jumps over the lazy dog',
        'Luke is the mechanical and electrical engineer of the group',
        'Instead of buying a new engine, buy a new car',
        'An engineer may design car engines of all sorts',
        'Engineers use logic, but not necessarily imagination',
        'Logic will take you from A to Z, imagination will take you everywhere.',
        'Continuous effort, not strength or intelligence, is the key to \
        unlocking our potential. And curiosity.',
        'It’s OK to have your eggs in one basket as long as you control what \
        happens to that basket.'
        ]

cleaned_docs = []
for doc in docs:
    to_app = clean_tokenize(doc)
    cleaned_docs.append(to_app)

print(index_one_doc(cleaned_docs[4],to_stem=True))
print(index_one_doc(cleaned_docs[4],to_stem=False))

{'an': [0], 'engin': [1, 5], 'may': [2], 'design': [3], 'car': [4], 'of': [6], 'all': [7], 'sort': [8]}
{'an': [0], 'engineer': [1], 'may': [2], 'design': [3], 'car': [4], 'engines': [5], 'of': [6], 'all': [7], 'sorts': [8]}


We can see that obviously, stemming affects our inverted index. In the example above, 'engineer' and 'engines' are collapsed to the same 'engin' term when stemming is used. Let's now construct the full inverted index. Note that we make the following choices: our (1) queries will not be case sensitive, (2) we are indexing punctuation marks, (3) we are indexing stopwords and thus can have more expressive queries (that capture negation, for instance). We also construct two inverted indexes: one using stemming and one not using stemming.

In [5]:
inverted_index = dict()
inverted_index_stem = dict()

for d_idx,doc in enumerate(cleaned_docs):
    
    poslists_s = index_one_doc(doc,to_stem=True) # get positions of each token in the doc
    for tok,poslist_s in poslists_s.items():
        if tok in inverted_index_stem:
            inverted_index_stem[tok][d_idx] = poslist_s # update
        else:
            inverted_index_stem[tok] = dict()
            inverted_index_stem[tok][d_idx] = poslist_s # initialize
    
    poslists = index_one_doc(doc,to_stem=False)
    for tok,poslist in poslists.items():
        if tok in inverted_index:
            inverted_index[tok][d_idx] = poslist
        else:
            inverted_index[tok] = dict()
            inverted_index[tok][d_idx] = poslist

print(inverted_index_stem)

{'the': {0: [0, 6], 1: [0, 6], 2: [2, 8], 7: [9]}, 'quick': {0: [1], 1: [2]}, 'brown': {0: [2], 1: [1]}, 'fox': {0: [3], 1: [3]}, 'jump': {0: [4], 1: [4]}, 'over': {0: [5], 1: [5]}, 'lazi': {0: [7], 1: [7]}, 'dog': {0: [8], 1: [8]}, 'luke': {2: [0]}, 'is': {2: [1], 7: [8]}, 'mechan': {2: [3]}, 'and': {2: [4], 7: [16]}, 'electr': {2: [5]}, 'engin': {2: [6], 3: [5], 4: [1, 5], 5: [0]}, 'of': {2: [7], 3: [1], 4: [6]}, 'group': {2: [9]}, 'instead': {3: [0]}, 'buy': {3: [2, 7]}, 'a': {3: [3, 8], 6: [5]}, 'new': {3: [4, 9]}, ',': {3: [6], 5: [3], 6: [8], 7: [2, 7]}, 'car': {3: [10], 4: [4]}, 'an': {4: [0]}, 'may': {4: [2]}, 'design': {4: [3]}, 'all': {4: [7]}, 'sort': {4: [8]}, 'use': {5: [1]}, 'logic': {5: [2], 6: [0]}, 'but': {5: [4]}, 'not': {5: [5], 7: [3]}, 'necessarili': {5: [6]}, 'imagin': {5: [7], 6: [9]}, 'will': {6: [1, 10]}, 'take': {6: [2, 11]}, 'you': {6: [3, 12], 8: [14]}, 'from': {6: [4]}, 'to': {6: [6], 7: [11], 8: [4, 18]}, 'z': {6: [7]}, 'everywher': {6: [13]}, '.': {6: [14

We can see that our full inverted index contains the positional lists of each term in each document of the collection.

## Querying
Let's now define some functions to execute word and phrase queries over our collection:

In [6]:
def at_least_one_unigram(query,inverted_index):
    '''
    returns the indexes of the docs containing *at least one* query unigrams
    the query is a list of unigrams
    '''
    
    to_return = []
    for unigram in query:
        if unigram in inverted_index:
            to_return.extend(list(inverted_index[unigram].keys()))
    return list(set(to_return))

def all_unigrams(query,inverted_index):
    '''
    returns the indexes of the docs containing *all* query unigrams
    the query is a list of unigrams
    '''
    
    to_return = []
    for unigram in query:
        if unigram in inverted_index:
            to_return.append(set(list(inverted_index[unigram].keys())))
        else:
            to_return.append(set())
            break
    to_return = to_return[0].intersection(*to_return)
    return list(to_return)

def ngrams(query,inverted_index):
    '''
    returns the indexes of the docs containing all unigrams in same order as the query
    the query is a list of unigrams
    '''
    candidate_docs = all_unigrams(query,inverted_index)   
    
    to_return = []
    for doc in candidate_docs:
        poslists = []
        for unigram in query:
            to_append = inverted_index[unigram][doc]
            if isinstance(to_append, int):
                poslists.append([to_append])
            else:
                poslists.append(to_append)
        # test whether the query words are consecutive    
        poslists_sub = [[elt-idx for elt in poslist] for idx,poslist in enumerate(poslists)]
        if set(poslists_sub[0]).intersection(*poslists_sub):
            to_return.append(doc)
    return to_return

Let's perform a few queries to illustrate how the functions work.

In [7]:
# 1) docs containing engine OR car
query = ['engine','car']
print(at_least_one_unigram(query,inverted_index))

# 2) docs containing engine AND car
print(all_unigrams(query,inverted_index))

# repeat 1) and 2) above, using stemming
query_stemmed = [stemmer.stem(elt) for elt in query]
print(at_least_one_unigram(query_stemmed,inverted_index_stem))
print(all_unigrams(query_stemmed,inverted_index_stem))

# 3) all unigrams vs. phrase query
query = ['quick','brown']
print(at_least_one_unigram(query,inverted_index))
print(ngrams(query,inverted_index))

[3, 4]
[3]
[2, 3, 4, 5]
[3, 4]
[0, 1]
[0]


Recall that our collection is:
```python
docs = ['The quick brown fox jumps over the lazy dog',
        'The brown quick fox jumps over the lazy dog',
        'Luke is the mechanical and electrical engineer of the group',
        'Instead of buying a new engine, buy a new car',
        'An engineer may design car engines of all sorts',
        'Engineers use logic, but not necessarily imagination',
        'Logic will take you from A to Z, imagination will take you everywhere.',
        'Continuous effort, not strength or intelligence, is the key to \
        unlocking our potential. And curiosity.',
        'It’s OK to have your eggs in one basket as long as you control what \
        happens to that basket.'
        ]
```

## TF-IDF
Let's now compute the IDF coefficients for our words. These coefficients are a measure of term specificity. They are computed as:

\begin{equation}
idf(t,D) = \log \big( \frac{m}{1+df(t)} \big)
\end{equation}

where $df(t)$ is the number of documents in the collection $D$ that contain $t$ and $m$ is the size of $D$. The IDF coefficients are usually then plugged in the equation below to compute feature vetors for the documents:

\begin{equation}
\mathrm{weight}(t,d,D) = tf(t,d) \times idf(t,D)
\end{equation}

Where $tf(t,d)$ is the number of times term $t$ appears in document $d$. Here, the documents are represented as **bags of words**.


The intuition behind bag-of-words + TF-IDF coefficients is that *frequent words in a document are representative of that document as long as they are not also very frequent at the corpus level*. 

In practice, the TF-IDF vectors can be used (1) in querying, to compute relevance scores for the documents, and (2) as features for (un)supervised tasks such as document clustering or classification.

In [9]:
all_unique_terms = list(set([elt for sublist in cleaned_docs for elt in sublist])) # flatten docs

terms_by_doc_sets = [set(elt) for elt in cleaned_docs]
n_doc = len(cleaned_docs)
idfs = dict(zip(all_unique_terms,[0]*len(all_unique_terms)))
dfs = dict(zip(all_unique_terms,[0]*len(all_unique_terms)))

for unique_term in list(idfs.keys()):
    df = sum([unique_term in terms for terms in terms_by_doc_sets]) # nb of docs in which 'unique_term' appears
    dfs[unique_term] = df
    idfs[unique_term] = round(math.log10(n_doc/(1+df)),5)

sorted_idfs = sorted(idfs.items(), key=operator.itemgetter(1), reverse=False)

# terms sorted in increasing order of specificity
print(sorted_idfs)

[('the', 0.25527), (',', 0.25527), ('.', 0.35218), ('of', 0.35218), ('to', 0.35218), ('lazy', 0.47712), ('car', 0.47712), ('is', 0.47712), ('fox', 0.47712), ('not', 0.47712), ('engineer', 0.47712), ('imagination', 0.47712), ('and', 0.47712), ('quick', 0.47712), ('over', 0.47712), ('dog', 0.47712), ('brown', 0.47712), ('jumps', 0.47712), ('a', 0.47712), ('logic', 0.47712), ('you', 0.47712), ('an', 0.65321), ('may', 0.65321), ('necessarily', 0.65321), ('new', 0.65321), ('curiosity', 0.65321), ('effort', 0.65321), ('everywhere', 0.65321), ('in', 0.65321), ('buying', 0.65321), ('strength', 0.65321), ('use', 0.65321), ('as', 0.65321), ('engines', 0.65321), ('engine', 0.65321), ('group', 0.65321), ('z', 0.65321), ('or', 0.65321), ('engineers', 0.65321), ('take', 0.65321), ('from', 0.65321), ('your', 0.65321), ('continuous', 0.65321), ('it', 0.65321), ('long', 0.65321), ('that', 0.65321), ('but', 0.65321), ('happens', 0.65321), ('electrical', 0.65321), ('sorts', 0.65321), ('buy', 0.65321), ('

Of course, we have a very small toy collection, so the picture is quite blurry here. But we can still see that the stopwords and punctuation marks have the lowest IDF scores (lowest specificity), which makes sense. Note that in a real corpus, there will be orders of magnitude differences between the IDF scores of the stopwords and that of the most specific words. Stopwords have very high TF values in all documents, but since their IDF scores are very low, their resulting final TF-IDF scores will be small.